# StupidNet1
StupidNet1 applies a convolutional layer and then a fully connected layer to both channels of the input data to predict the binary target. This is done without any auxiliary losses based on the digit classes of the two channels. With the current architecture we can clearly see overfitting.

In [ ]:
import torch
import matplotlib.pyplot as plt
from torch import nn
from torch import optim
import dlc_practical_prologue as prologue

In [ ]:
def normalize_data(data):
    mu, std = data[0].mean(), data[0].std()
    
    data[0].sub_(mu).div_(std)
    data[3].sub_(mu).div_(std)
    
    return data

In [ ]:
pair_data = prologue.generate_pair_sets(1000)
pair_data = normalize_data(pair_data)

In [ ]:
class StupidNet1(nn.Module):
    def __init__(self, nb_hidden):
        super(StupidNet1, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(2, 32, kernel_size=3),         # 32 x 12 x 12
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=3),   # 32 x 4 x 4
            nn.Conv2d(32, 64, kernel_size=3),        # 64 x 2 x 2
            nn.ReLU(inplace=True),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(64 * 2 * 2, nb_hidden),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(nb_hidden, 2),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, 64 * 2 * 2)
        x = self.classifier(x)
        return x  

In [ ]:
def train_model(model, train_input, train_target):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr = 1e-1)
    nb_epochs = 100

    for e in range(nb_epochs):
        sum_loss = 0
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            model.zero_grad()
            loss.backward()
            optimizer.step()
            sum_loss = sum_loss + loss.item()
        print(e, sum_loss)

In [ ]:
def compute_nb_errors(model, data_input, data_target):

    nb_data_errors = 0

    for b in range(0, data_input.size(0), mini_batch_size):
        output = model(data_input.narrow(0, b, mini_batch_size))
        _, predicted_classes = torch.max(output, 1)
        for k in range(mini_batch_size):
            if data_target.data[b + k] != predicted_classes[k]:
                nb_data_errors = nb_data_errors + 1

    return nb_data_errors

In [ ]:
stupidNet1 = StupidNet1(100);

mini_batch_size = 100;

train_model(stupidNet1, pair_data[0], pair_data[1])

In [ ]:
nb_train_errors = compute_nb_errors(stupidNet1, pair_data[0], pair_data[1])
nb_test_errors = compute_nb_errors(stupidNet1, pair_data[3], pair_data[4])
print('train error Net {:0.2f}% {:d}/{:d}'.format((100 * nb_train_errors) / pair_data[0].size(0),
                                                  nb_train_errors, pair_data[0].size(0)))
print('test error Net {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / pair_data[4].size(0),
                                              nb_test_errors, pair_data[4].size(0)))
